# Class Weight



## 0.환경준비

### 0.1 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### 0.2 데이터 로딩

> 예제 데이터는 공정간 불량을 예측하는 데이터입니다.  
여기서는 class imbalance 에 대한 샘플링과 성능에만 초점을 맞추겠습니다.


In [ ]:
# data data
path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/Attrition2.csv"
data = pd.read_csv(path)

## 1.데이터 탐색

In [ ]:
data.shape

In [ ]:
data.head()

## 2.데이터 준비

### 2.1 x, y 로 나누기

In [ ]:
target = 'Attrition'
x = data.drop(target, axis = 1)
y = data.loc[:,target]

### 2.2 가변수화


In [ ]:
dummy_vars = ['Education','Department','EducationField','Gender','JobRole','JobSatisfaction','MaritalStatus',
              'RelationshipSatisfaction','WorkLifeBalance'] # 가변수화 대상 변수 이름 리스트 만든다
x = pd.get_dummies(x, columns = dummy_vars, drop_first=True) 

### 2.3 데이터 분할

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = .3, random_state=2022)

In [ ]:
x_train.shape, y_train.shape

## 3.모델링 + class weight 조정하기

In [ ]:
# 필요한 라이브러리, 함수들을 불러옵시다.
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import *

from sklearn.model_selection import GridSearchCV, StratifiedKFold

### 1) 기본 모델링

* 로지스틱 회귀 + 데이터 그대로 이용

In [ ]:
model = LogisticRegression(solver = 'liblinear')
model.fit(x_train, y_train)
pred = model.predict(x_val)

print(confusion_matrix(y_val, pred))
print('-' * 55)
print(classification_report(y_val, pred))

### 2) Class Weight 조정

In [ ]:
def loss(y, y_hat):
    loss = -np.mean(y*(np.log(y_hat)) - (1-y)*np.log(1-y_hat))
    return loss



#### ① 로지스틱 회귀
> * class_weight = 'balanced'
* class_weight = 비율로 수동 조절

In [ ]:
# class_weight = 'balanced'
 
model = LogisticRegression(solver = 'liblinear', class_weight='balanced')
model.fit(x_train, y_train)
pred = model.predict(x_val)

print(confusion_matrix(y_val, pred))
print('-' * 55)
print(classification_report(y_val, pred))

In [ ]:
result = []

for i in range(50):
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = .4)
    model = LogisticRegression(solver = 'liblinear', class_weight='balanced')
    model.fit(x_train, y_train)
    pred = model.predict(x_val)
    result.append(f1_score(y_val, pred, pos_label=1))
    print(i)

In [ ]:
plt.figure(figsize=(12, 8))
sns.kdeplot(result)
plt.title('F1 Score')
plt.grid()
plt.show()

In [ ]:
# class_weight = 비율로 수동 조절하도록 파라미터 설정
weights = np.linspace(0.0,.99,100)
params = {'class_weight': [{0:x, 1:1.0-x} for x in weights]}
params

In [ ]:
# Grid Search : 그리드서치로 튜닝
# StratifiedKFold : 층화추출
model = GridSearchCV(LogisticRegression(solver = 'liblinear'), params
                    , cv= StratifiedKFold(5), scoring='f1')
model.fit(x_train, y_train)

In [ ]:
y_train.value_counts()/y_train.shape[0]

In [ ]:
model.cv_results_

In [ ]:
weight_1 = 1.0 - weights
f1 = model.cv_results_['mean_test_score']

print(model.best_params_)
plt.figure(figsize=(12, 8))
plt.plot(weight_1, f1)
plt.ylabel('F1 Score')
plt.xlabel('Class 1 weight')
plt.grid()
plt.show()

#### ③ 추가 실험

* 이해하기, 사용하기 쉬운 데이터로 몇가지 실험을 해 봅시다.

In [ ]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/titanic.1.csv'
titanic = pd.read_csv(path, usecols=['Survived', 'Age', 'Fare'])
target = 'Survived'
x1 = titanic.drop(target, axis = 1)
y1 = titanic.loc[:, target]

In [ ]:
# 클래스별 비율
c_rate = np.bincount(y1) / y1.shape[0]
c_rate

In [ ]:
# 실험1 : 아무것도 없으면 1:1 ?
m1 = LogisticRegression(solver = 'liblinear')
wt = 0.5
m2 = LogisticRegression(solver = 'liblinear', class_weight= {0:wt, 1:1-wt})

# 실험2 : balanced는 클래스비율 역으로 지정?
m3 = LogisticRegression(solver = 'liblinear', class_weight='balanced')
wt = c_rate[1]
m4 = LogisticRegression(solver = 'liblinear', class_weight= {0:wt, 1:1-wt})

m1.fit(x1, y1)
m2.fit(x1, y1)
m3.fit(x1, y1)
m4.fit(x1, y1)

In [ ]:
print(list(x1))
print('-'*50)
print(m1.coef_, m1.intercept_)
print(m2.coef_, m2.intercept_)
print('-'*50)
print(m3.coef_, m3.intercept_)
print(m4.coef_, m4.intercept_)

#### ② Decision Tree
> * class_weight = 'balanced'
* class_weight = 비율로 수동 조절

In [ ]:
# 실험1 
m1 = DecisionTreeClassifier(max_depth = 2)

wt = 0.5
m2 = DecisionTreeClassifier(max_depth = 2, class_weight= {0:wt, 1:1-wt})

m1.fit(x1, y1)
m2.fit(x1, y1)

plt.figure(figsize = (10,6))
plot_tree(m1, feature_names = list(x1), class_names = ['Died', 'Survived'], rounded = True, filled = True)
plt.show()
plt.figure(figsize = (10,6))
plot_tree(m2, feature_names = list(x1), class_names = ['Died', 'Survived'], rounded = True, filled = True)
plt.show()

In [ ]:
c_rate = np.bincount(y1) / y1.shape[0]
c_rate

In [ ]:
# 실험2 : balanced
m3 = DecisionTreeClassifier(max_depth = 2, class_weight='balanced')

wt = c_rate[1]
m4 = DecisionTreeClassifier(max_depth = 2, class_weight= {0:wt, 1:1-wt})

m3.fit(x1, y1)
m4.fit(x1, y1)

plt.figure(figsize = (10,6))
plot_tree(m3, feature_names = list(x1), class_names = ['Died', 'Survived'], rounded = True, filled = True)
plt.show()
plt.figure(figsize = (10,6))
plot_tree(m4, feature_names = list(x1), class_names = ['Died', 'Survived'], rounded = True, filled = True)
plt.show()

## 4.실습 : Semiconductor manufacturing process dataset


![](https://assets.pandaily.com/uploads/2021/10/semiconductor.png)

* 반도체 제조 공정은 시점별로 수많은 센서로부터 정보를 수집하여 공정을 감시합니다. 
* 센서정보와 함께 공정간 발생된 불량품에 대한 정보를 저장하였습니다.
* 불량을 예측해 봅시다.

### 4.1 데이터 준비

* 데이터 로딩

In [ ]:
path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/secom_9.csv"
data = pd.read_csv(path)

data['label'] = 0
data.loc[data['defeat']== 'defeat', 'label']= 1
data.drop(['datetime','defeat'], axis = 1, inplace=True)
data.head()

변수 정보 
* label : 1 - 불량, 0 - 정상
* v### : 센서값들


In [ ]:
target = 'label'

In [ ]:
data[target].value_counts() / data.shape[0]

* x, y로 나누기 

In [ ]:
x = data.drop(target, axis = 1)
y = data.loc[:, target]

* 가변수화 Dummy Variable

* 데이터 분할
    * 이미 test set은 분할되어 있다고 가정합니다.
    * 주어진 데이터를 train set : validation set 으로 분할

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# train_val에서 train : val = 8 : 2
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state = 2022)

In [ ]:
print(x_train.shape, x_val.shape)

### 2)기본모델링
> * 데이터셋을 그대로 모델링을 수행하시오.
* 알고리즘 : 로지스틱회귀, decision tree 중 하나 사용.
* validate set으로 예측하고 평가(classification report, f1 score)해 봅시다.


In [ ]:
model = LogisticRegression(solver = 'liblinear')
model.fit(x_train, y_train)
pred = model.predict(x_val)

print(confusion_matrix(y_val, pred))
print('-' * 55)
print(classification_report(y_val, pred))

### 2)모델링 + class weight 조정
> * class_weight 값을 지정해서 모델링 해 봅시다.
* 알고리즘 : 비교를 위해서 기본모델링에 사용한 알고리즘 사용
* validate set으로 예측하고 평가(classification report, f1 score)해 봅시다.
* resampling 후 수행한 모델과 성능에 대해 비교해 봅시다.

In [ ]:
# class_weight = 'balanced'
 


* 로지스틱 회귀로 반복 실험

* GridSearch로 class weight 조절